# Reflection (自我反思) 智能体实战

本笔记本演示了 **Reflection (自我反思)** 模式。这种模式赋予了智能体“自我批判”和“迭代修复”的能力。

### 核心思想：
1. **生成 (Worker)**：根据要求初步完成任务。
2. **反思 (Reviewer)**：对自己生成的结果进行审查，找出不足或错误。
3. **修正 (Refiner)**：根据反思的意见进行自我迭代和优化。

### C/C++ 类比：
这非常像 **“自测 -> 修复 Bug”** 的开发循环：
- **生成**：编写初版 C++ 函数（可能是暴力破解法）。
- **反思**：进行 **Code Review**，发现算法复杂度太高（例如 $O(n^2)$），或者存在内存泄漏隐患。
- **修正**：根据 Review 意见重构代码，优化算法（例如改进为 $O(n \log n)$）。
- 用户设置 `max_iterations` 就像是设定了 **“重构轮数上限”**。

In [1]:
import os
from dotenv import load_dotenv
from typing import List, Dict, Any
from llm_client import HelloAgentsLLM

# 1. 环境准备
load_dotenv(dotenv_path="../../.env")
llm_client = HelloAgentsLLM()

In [2]:
# --- 2. 记忆模块 (Memory) ---

class Memory:
    """
    短期记忆模块，类比 C++ 的 std::vector<ExecutionRecord>，记录演进轨迹。
    """
    def __init__(self):
        self.records: List[Dict[str, Any]] = []

    def add_record(self, record_type: str, content: str):
        self.records.append({"type": record_type, "content": content})
        print(f"📝 记忆记录: 新增 '{record_type}'。")

    def get_last_execution(self) -> str:
        for record in reversed(self.records):
            if record['type'] == 'execution':
                return record['content']
        return None

In [3]:
# --- 3. 提示词模板 (Prompt Templates) ---

INITIAL_PROMPT_TEMPLATE = """
你是一位资深的Python程序员。请根据以下要求，编写一个Python函数。
要求: {task}
直接输出代码。
"""

REFLECT_PROMPT_TEMPLATE = """
你是一位极其严格的代码评审专家。分析以下代码的时间复杂度并提出改进建议。
# 原始任务: {task}
# 待审查代码:
```python
{code}
```
如有改进空间请指出具体的改进算法建议。如果已达到最优，请回答“无需改进”。
"""

REFINE_PROMPT_TEMPLATE = """
你正在根据评审反馈优化代码。
# 原始任务: {task}
# 上一轮代码: {last_code_attempt}
# 评审反馈: {feedback}
请生成优化后的新代码。
"""

In [4]:
# --- 4. Reflection 智能体实现 ---

class ReflectionAgent:
    def __init__(self, llm_client, max_iterations=2):
        self.llm_client = llm_client
        self.memory = Memory()
        self.max_iterations = max_iterations

    def run(self, task: str):
        # 1. 初始生成 (Initial Attempt)
        print("\n--- 初始生成阶段 ---")
        initial_code = self.llm_client.think([{"role": "user", "content": INITIAL_PROMPT_TEMPLATE.format(task=task)}])
        self.memory.add_record("execution", initial_code)

        # 2. 迭代：反思 -> 优化 (Reflect-Refine Loop)
        # 类比 C++ 的 while(attempts < max && !optimized) 循环
        for i in range(self.max_iterations):
            print(f"\n--- 第 {i+1} 轮反思与优化 ---")
            
            # a. 进行 Code Review
            last_code = self.memory.get_last_execution()
            feedback = self.llm_client.think([{"role": "user", "content": REFLECT_PROMPT_TEMPLATE.format(task=task, code=last_code)}])
            self.memory.add_record("reflection", feedback)

            if "无需改进" in feedback:
                print("✅ 达成最优状态。")
                break

            # b. 执行重构
            refined_code = self.llm_client.think([{"role": "user", "content": REFINE_PROMPT_TEMPLATE.format(task=task, last_code_attempt=last_code, feedback=feedback)}])
            self.memory.add_record("execution", refined_code)
        
        return self.memory.get_last_execution()

# --- 测试运行 ---
agent = ReflectionAgent(llm_client)
# 故意给出一个可以让它优化的任务（先用试除法，再反思出筛法）
task = "编写一个获取 100 以内所有质数的函数"
final_result = agent.run(task)
print("\n--- 最终输出 ---")
print(final_result)


--- 初始生成阶段 ---
🧠 正在调用 gpt-4o 模型...
✅ 大语言模型响应成功:
```python
def get_primes_upto_100():
    primes = []
    for num in range(2, 101):  # Start from 2, as 1 is not a prime number
        is_prime = True
        for i in range(2, int(num ** 0.5) + 1):  # Check divisors up to the square root of the number
            if num % i == 0:
                is_prime = False
                break
        if is_prime:
            primes.append(num)
    return primes

# Example usage
print(get_primes_upto_100())
```
📝 记忆记录: 新增 'execution'。

--- 第 1 轮反思与优化 ---
🧠 正在调用 gpt-4o 模型...
✅ 大语言模型响应成功:
### 时间复杂度分析

1. **外层循环**: `for num in range(2, 101)`  
   - 这个循环遍历从 2 到 100 的所有整数，共计 99 次。

2. **内层循环**: `for i in range(2, int(num ** 0.5) + 1)`  
   - 对于每个 `num`，内层循环检查从 2 到 `sqrt(num)` 的所有整数是否是 `num` 的因子。  
   - 最坏情况下，内层循环的迭代次数为 `O(sqrt(num))`。  
   - 因此，内层循环的总复杂度为 `O(√2 + √3 + √4 + ... + √100)`。

3. **整体复杂度**:  
   - 由于平方根的增长速度较慢，内层循环的复杂度可以近似为 `O(n * sqrt(n))`，其中 `n` 是外层循环的范围（即 100）。  
   - 因此，整体时间复杂度为 **O(n *